In [ ]:
%matplotlib inline
import os

import numpy as np
import pandas as pd
import scipy.ndimage
import matplotlib.pyplot as plt

import pydicom
import nrrd

# DICOM - the data

In [1]:
scan_number = '23262134'

In [2]:
!ls ../data/scans/{scan_number}/ | head -n 3

23262134
23262145
23262156


In [3]:
!ls ../data/scans/{scan_number}/ | wc -l

     626


In [4]:
!ls ../data/masks/

23262134.nrrd


In [ ]:
def load_scan(path, use_even=False):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    if use_even:
        return slices[::2]
    return slices

In [ ]:
def get_pixels_hu(slices):
    pixel_arrays = [s.pixel_array for s in slices]
    
    image = np.stack(pixel_arrays)
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

<img src="https://pbrainmd.files.wordpress.com/2015/10/hounsfield-2.jpg">

In [ ]:
scan = load_scan(f'../data/scans/{scan_number}', use_even=True)

In [ ]:
len(scan)

In [ ]:
scan[0]

In [ ]:
scan_pixels = get_pixels_hu(scan)
plt.hist(scan_pixels.flatten(), bins=80, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

# Show some slice in the middle
plt.imshow(scan_pixels[150], cmap=plt.cm.gray)
plt.show()

In [ ]:
scan_pixels.shape

## Adjust grayscale

In [ ]:
scan_pixels_copy = scan_pixels.copy()
mask = (scan_pixels_copy >= -50) & (scan_pixels_copy <= 150)

scan_pixels_copy[mask] = np.interp(
    scan_pixels_copy[mask], 
    (scan_pixels_copy[mask].min(), scan_pixels_copy[mask].max()), 
    (0, 6000),
)
scan_pixels_copy[~mask] = -1000

In [ ]:
for i in range(130, len(scan_pixels) - 50, 8):
    plt.imshow(scan_pixels_copy[i], cmap=plt.cm.gray)
    plt.show()
#     fig = plt.figure(figsize=(10,10))
#     ax1 = fig.add_subplot(1,2,1)
#     ax1.imshow(scan_pixels_copy[i], cmap=plt.cm.gray)
#     ax2 = fig.add_subplot(1,2,2)
#     ax2.imshow(segmentation[i], cmap=plt.cm.Reds)
#     plt.show()

# NRRD - segmenation

In [ ]:
segmentation, metadata = nrrd.read(f'../data/masks/{scan_number}.nrrd')

In [ ]:
segmentation.shape

In [ ]:
metadata

## Make axis consistent

In [ ]:
segmentation = np.rollaxis(np.rollaxis(segmentation, 1), 2)

## Show segmeneted 

In [ ]:
for i in range(130, len(scan_pixels) - 50, 10):
    plt.imshow(scan_pixels[i], cmap=plt.cm.gray)
    plt.imshow(segmentation[i], cmap=plt.cm.Reds, alpha=0.2, vmin=0, vmax=1)
    plt.show()

# Segmented & mask side by side

In [ ]:
for i in range(130, len(scan_pixels) - 50, 8):
    fig = plt.figure(figsize=(10,10))
    ax1 = fig.add_subplot(1,2,1)
    ax1.imshow(scan_pixels_copy[i], cmap=plt.cm.gray)
    ax2 = fig.add_subplot(1,2,2)
    ax2.imshow(segmentation[i], cmap=plt.cm.Reds)
    plt.show()

# Make stuff machine learning friendly - spacing

In [ ]:
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing = np.array([scan[0].SliceThickness] + list(scan[0].PixelSpacing), dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image

In [ ]:
scan_pixels = resample(scan_pixels, scan, [1, 1, 1])
segmentation = resample(segmentation, scan, [1, 1, 1])

In [ ]:
scan_pixels.shape

In [ ]:
for i in range(130, len(scan_pixels) - 50, 10):
    plt.imshow(scan_pixels[i], cmap=plt.cm.gray)
    plt.imshow(segmentation[i], cmap=plt.cm.Reds, alpha=0.2, vmin=0, vmax=1)
    plt.show()